# Nationalratswahlen seit 1995

Quellen:

- **NRW 1995:** https://www.bmi.gv.at/412/Nationalratswahlen/Nationalratswahl_1995_Wiederholungswahl_1996/start.aspx
- **NRW 1999:** https://www.bmi.gv.at/412/Nationalratswahlen/Nationalratswahl_1999/start.aspx
- **NRW 2002:** https://www.bmi.gv.at/412/Nationalratswahlen/Nationalratswahl_2002/start.aspx
- **NRW 2006:** https://www.bmi.gv.at/412/Nationalratswahlen/Nationalratswahl_2006/start.aspx
- **NRW 2008:** https://www.bmi.gv.at/412/Nationalratswahlen/Nationalratswahl_2008/start.aspx
- **NRW 2013:** https://www.bmi.gv.at/412/Nationalratswahlen/Nationalratswahl_2013/start.aspx
- **NRW 2017:** https://www.bmi.gv.at/412/Nationalratswahlen/Nationalratswahl_2017/start.aspx
- **NRW 2019:** https://www.bmi.gv.at/412/Nationalratswahlen/Nationalratswahl_2019/start.aspx
- **Umcodierung der Gemeindekennziffern Niederösterreich:** https://www.data.gv.at/katalog/dataset/7f759371-bdb3-4f58-9dbd-835eb3c19efa
- **Die neue Gemeinde- und Bezirksstruktur in der Steiermark:** https://www.landesentwicklung.steiermark.at/cms/beitrag/12658686/141979478/
- **Land Steiermark: Gebietsstands- und Namensänderungen 2020:** https://www.landesentwicklung.steiermark.at/cms/dokumente/12658757_142970621/716b52c0/STMK%20Gebietsstands%C3%A4nderungen%20GM%202020.pdf

In [ ]:
import pandas as pd, datetime as dt, numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', 128)

### Lesen der Verwaltungsgliederung und Einpflegen der Änderungen

In [ ]:
gemeinden = pd.read_csv("gemeinden.tsv", encoding="utf-8", sep="\t",
                        dtype={"GKZ": int, "BKZ": int, "BLKZ": int, "PG": "string", "PB": "string",
                               "BL": "string", "WKNR": "string", "WKNAME": "string", "AREA": float,
                               "CENTER_X": float, "CENTER_Y": float}) \
    .set_index("GKZ", verify_integrity=True) \
    .rename({"BKZ": "BKZ_BEV", "WKNR": "WKNR_BEV"}, axis=1)
# Gebietsreform in der Steiermark (sehr umfassend)
steiermark_bez_neu_2013_replace = pd.read_csv("steiermark_bez_neu_2013.tsv", encoding="utf-8", sep="\t",
                                              usecols=["GKZ_NEU", "GKZ_ALT"],
                                              dtype={"GKZ_ALT": int, "GKZ_NEU": int}) \
    .set_index("GKZ_ALT", verify_integrity=True).GKZ_NEU.to_dict()
# Manche Gemeinden werden Geteilt. Wir weisen sie aber dem letzten Eintrag zu.
steiermark_gkz_neu_2015_replace = pd.read_csv("steiermark_gkz_neu_2015.tsv", encoding="utf-8", sep="\t",
                                              usecols=["GKZ_NEU", "GKZ_ALT"],
                                              dtype={"GKZ_ALT": int, "GKZ_NEU": int}) \
    .set_index("GKZ_ALT").GKZ_NEU.to_dict()
steiermark_gkz_neu_2020_replace = pd.read_csv("steiermark_gkz_neu_2020.tsv", encoding="utf-8", sep="\t",
                                              usecols=["GKZ_NEU", "GKZ_ALT"],
                                              dtype={"GKZ_ALT": int, "GKZ_NEU": int}) \
    .set_index("GKZ_ALT").GKZ_NEU.to_dict()
# Auflösung des Bezirkes Wien Umgebung in NÖ
noe_neu_replace = pd.read_csv("noe_umcodierung_lau2.csv", sep=";", encoding="utf_8",
                      usecols=["LAU2_CODE_2016", "LAU2_CODE_2017"],
                      dtype={"LAU2_CODE_2016": int, "LAU2_CODE_2017": int})
noe_neu_replace = noe_neu_replace[noe_neu_replace.LAU2_CODE_2016 != noe_neu_replace.LAU2_CODE_2017] \
    .set_index("LAU2_CODE_2016", verify_integrity=True).LAU2_CODE_2017.to_dict()
# Manuell eingepflegte Änderungen aus Gesetzesblättern, etc.
gemeinden_neu_replace = pd.read_csv("gemeinden_aenderungen_manuell.tsv", sep="\t", encoding="utf_8",
                            usecols=["GKZ_NEU", "GKZ_ALT"],
                            dtype={"GKZ_ALT": int, "GKZ_NEU": int}) \
    .set_index("GKZ_ALT", verify_integrity=True).GKZ_NEU.to_dict()


### Lesen der Ergebnisse

In [ ]:
files = {"nrw1995.tsv": 1995,"nrw1999.tsv": 1999,"nrw2002.tsv": 2002,"nrw2006.tsv": 2006,
         "nrw2008.tsv": 2008,"nrw2013.tsv": 2013,"nrw2017.tsv": 2017,"nrw2019.tsv": 2019}

In [ ]:
results = None
for file, year in files.items():
    nrw = pd.read_csv(file, encoding="utf-8", sep="\t",
                      dtype={"GKZ": "string", "Gebiet": "string", "Wahlberechtigte": "Int64",
                             "StimmenAbgegeben": "Int64", "StimmenGueltig": "Int64", "StimmenUngueltig": "Int64"})
    nrw["WKNR"] = nrw.GKZ.str.extract(r"^(\d[A-Z])\d\d\d$")
    nrw["BKZ"] = nrw.GKZ.str.extract(r"^(\d\d\d)\d\d$").astype("Int64")
    nrw["GKZ_BEV"] = np.where(nrw.GKZ.str.match(r"^[0-9]{5}$") & ~nrw.GKZ.str.endswith("99") & ~nrw.GKZ.str.endswith("00"), nrw.GKZ, 0) \
        .astype(int)
    nrw["GKZ_BEV"] = nrw.GKZ_BEV.replace(steiermark_bez_neu_2013_replace)
    nrw["GKZ_BEV"] = nrw.GKZ_BEV.replace(steiermark_gkz_neu_2015_replace)
    nrw["GKZ_BEV"] = nrw.GKZ_BEV.replace(steiermark_gkz_neu_2020_replace)
    nrw["GKZ_BEV"] = nrw.GKZ_BEV.replace(noe_neu_replace)
    nrw["GKZ_BEV"] = nrw.GKZ_BEV.replace(gemeinden_neu_replace)
    gemeinden_ungueltig = nrw[nrw.GKZ_BEV != 0].join(gemeinden, on="GKZ_BEV") \
        .query("PG.isna()")[["GKZ", "GKZ_BEV", "Gebiet"]]
    if (len(gemeinden_ungueltig) > 0):
        print(f"Gemeindekennzahlen in der Datei {file} wurde nicht gefunden:")
        print(gemeinden_ungueltig)
    nrw["YEAR"] = year
    nrw = nrw.join(gemeinden, on="GKZ_BEV").astype({"BKZ_BEV": "Int64", "BLKZ": "Int64"})
    nrw = pd.melt(nrw.loc[:, ~nrw.columns.isin(["Gebiet"])],
                  id_vars=["GKZ", "GKZ_BEV", "WKNR", "BKZ", "Wahlberechtigte", "StimmenAbgegeben",
                           "StimmenGueltig", "StimmenUngueltig", "YEAR", "BKZ_BEV", "BLKZ", "PG", "PB",
                           "BL", "WKNR_BEV", "WKNAME", "AREA", "CENTER_X", "CENTER_Y"],
                  var_name="Partei", value_name="Stimmen").astype({"YEAR": int, "Partei": "string", "Stimmen": int})
    results = nrw if results is None else pd.concat([results, nrw])

#### Wahlkarten und Briefwahlkarten

Wahlkarten werden auf Wahlkreisebene erfasst.
Seit der NRW2008 gibt es die Möglichkeit der Briefwahl, sie wird bei den Bezirksergebnissen erfasst.
Beide werden für die weitere Bearbeitung aliquot nach den Wahlberechtigten aufgeteilt.

In [ ]:
wahlberechtigte_aliquot = results[results.GKZ_BEV != 0].astype({"GKZ": int, "BKZ": int, "Wahlberechtigte": int})\
    .groupby(["YEAR", "GKZ"], as_index=False).first() \
    [["YEAR", "GKZ", "BKZ", "WKNR_BEV", "Wahlberechtigte"]]
# Bis 2006 aggergieren wir über Wahlkreise, danach wird auf Bezirksebene aliquotiert
wahlberechtigte_aliquot["GemeindeanteilWahlkreis"] = wahlberechtigte_aliquot.Wahlberechtigte / wahlberechtigte_aliquot.groupby(["YEAR", "WKNR_BEV"]).Wahlberechtigte.transform(np.sum)
wahlberechtigte_aliquot["GemeindeanteilBezirk"] = wahlberechtigte_aliquot.Wahlberechtigte / wahlberechtigte_aliquot.groupby(["YEAR", "BKZ"]).Wahlberechtigte.transform(np.sum)
wahlberechtigte_aliquot = wahlberechtigte_aliquot.set_index(["YEAR", "GKZ"]) \
    [["GemeindeanteilWahlkreis", "GemeindeanteilBezirk"]]

In [ ]:
wahlkarten = results[results.GKZ.str.match(r"[0-9][A-Z][0-9]99$")] \
    ["WKNR YEAR Partei StimmenAbgegeben	StimmenGueltig	StimmenUngueltig Stimmen".split()] \
    .rename({"StimmenAbgegeben": "StimmenAbgegebenWahlkarten", "StimmenGueltig": "StimmenGueltigWahlkarten",
             "StimmenUngueltig": "StimmenUngueltigWahlkarten", "Stimmen": "StimmenWahlkarten"}, axis=1)
wahlkarten = wahlkarten.set_index(["YEAR", "WKNR", "Partei"])
wahlkarten = wahlkarten.astype(int)
briefwahl = results[results.GKZ.str.match(r"[0-9]{3}99$")].astype({"BKZ": int}) \
    ["BKZ YEAR Partei StimmenAbgegeben	StimmenGueltig	StimmenUngueltig Stimmen".split()] \
    .rename({"StimmenAbgegeben": "StimmenAbgegebenBriefwahl", "StimmenGueltig": "StimmenGueltigBriefwahl",
             "StimmenUngueltig": "StimmenUngueltigBriefwahl", "Stimmen": "StimmenBriefwahl"}, axis=1)
briefwahl = briefwahl.set_index(["YEAR", "BKZ", "Partei"]).astype(int)

In [ ]:
results_mit_wk = results[results.GKZ_BEV != 0] \
    .astype({'GKZ': int, 'BKZ': int, 'BKZ_BEV': int, 'BLKZ': int, 'Wahlberechtigte': int, 'StimmenAbgegeben': int, 'StimmenGueltig': int, 'StimmenUngueltig': int}) \
    .join(wahlkarten, on=["YEAR", "WKNR_BEV", "Partei"]) \
    .join(briefwahl, on=["YEAR", "BKZ", "Partei"]) \
    .join(wahlberechtigte_aliquot, on=["YEAR", "GKZ"])
results_mit_wk["StimmenAbgegebenMitWk"] = results_mit_wk.StimmenAbgegeben \
    + results_mit_wk.StimmenAbgegebenWahlkarten.fillna(0) * results_mit_wk.GemeindeanteilWahlkreis \
    + results_mit_wk.StimmenAbgegebenBriefwahl.fillna(0) * results_mit_wk.GemeindeanteilBezirk
results_mit_wk["StimmenGueltigMitWk"] = results_mit_wk.StimmenGueltig \
    + results_mit_wk.StimmenGueltigWahlkarten.fillna(0) * results_mit_wk.GemeindeanteilWahlkreis \
    + results_mit_wk.StimmenGueltigBriefwahl.fillna(0) * results_mit_wk.GemeindeanteilBezirk
results_mit_wk["StimmenUngueltigMitWk"] = results_mit_wk.StimmenUngueltig \
    + results_mit_wk.StimmenUngueltigWahlkarten.fillna(0) * results_mit_wk.GemeindeanteilWahlkreis \
    + results_mit_wk.StimmenUngueltigBriefwahl.fillna(0) * results_mit_wk.GemeindeanteilBezirk
results_mit_wk["StimmenMitWk"] = results_mit_wk.Stimmen \
    + results_mit_wk.StimmenWahlkarten.fillna(0) * results_mit_wk.GemeindeanteilWahlkreis \
    + results_mit_wk.StimmenBriefwahl.fillna(0) * results_mit_wk.GemeindeanteilBezirk
# Durch Gemeindezusammenlegungen gibt es mehrere Gemeindedatensätze pro Partei und Jahr. Diese
# werden zusammengefasst.
results_mit_wk = results_mit_wk[['YEAR', 'GKZ_BEV', 'BKZ_BEV', 'BLKZ', 'PG', 'PB',
    'BL', 'WKNR_BEV', 'WKNAME', 'AREA', 'CENTER_X', 'CENTER_Y',
    'Wahlberechtigte', 'StimmenAbgegeben', 'StimmenGueltig', 'StimmenUngueltig',
    'Partei', 'Stimmen', 'StimmenAbgegebenMitWk', 'StimmenGueltigMitWk', 'StimmenUngueltigMitWk',
    'StimmenMitWk']] \
        .groupby(['YEAR', 'GKZ_BEV', 'Partei'], as_index=False) \
        .aggregate({'BKZ_BEV': "first", 'BLKZ': "first", 'PG': "first", 'PB': "first", 'BL': "first",
                   'WKNR_BEV': "first", 'WKNAME': "first", 'AREA': "first", 'CENTER_X': "first", 'CENTER_Y': "first",
                   'Wahlberechtigte': "sum", 'StimmenAbgegeben': "sum", 'StimmenGueltig': "sum", 'StimmenUngueltig': "sum",
                   'Stimmen': "sum", 'StimmenAbgegebenMitWk': "sum", 'StimmenGueltigMitWk': "sum", 'StimmenUngueltigMitWk': "sum",
                   'StimmenMitWk': "sum"})

In [ ]:
results_mit_wk[results_mit_wk.YEAR == 1995].groupby(["GKZ_BEV"]).first().groupby(["BL"]) \
    .aggregate({'Wahlberechtigte': "sum", 'StimmenAbgegebenMitWk': "sum", 'StimmenGueltigMitWk': "sum", 'StimmenUngueltigMitWk': "sum"}) \
    .round(3)

### Schreiben der Ergebnisdatei

In [ ]:
parteien = pd.read_csv("parteien.tsv", encoding="utf-8", sep="\t",
                       dtype={"Jahr": int, "Parteibezeichnung": "string", "Kurzbezeichnung": "string"}) \
    .set_index(["Jahr", "Kurzbezeichnung"], verify_integrity=True)

In [ ]:
results_export = results_mit_wk.join(parteien, on=["YEAR", "Partei"])
pd.io.clipboards.to_clipboard(results_export.sample(5).to_markdown(index=False), excel=False)    # pip install tabulate --upgrade
results_export.to_csv("../nrw_ergebnisse.csv.bz2", compression={'method': 'bz2', 'compresslevel': 9}, sep=";", encoding="utf-8", index=False)
results_export.to_csv("../nrw_ergebnisse_unicode.csv.bz2", compression={'method': 'bz2', 'compresslevel': 9}, sep=";", encoding="utf-16", index=False)
results_export.to_parquet("../nrw_ergebnisse.parquet", compression="brotli")

In [ ]:
results_export.dtypes